In [60]:
import os
import io
import PIL.Image, PIL.ImageDraw
import base64
import zipfile
import json
import requests
import numpy as np
import matplotlib.pylab as pl
import glob

import tensorflow as tf

from tensorflow.keras.layers import Conv2D


In [61]:
#### UTILS #####
def get_living_mask(x):
    alpha = x[:, :, :, 3:4]
    return tf.nn.max_pool2d(alpha, 3, [1, 1, 1, 1], 'SAME') > 0.1

In [62]:
class CAModel(tf.keras.Model):
    
    def __init__(self, channel_n, fire_rate):
        super().__init__()
        self.channel_n = channel_n
        self.fire_rate = fire_rate

        self.dmodel = tf.keras.Sequential([
              Conv2D(128, 1, activation=tf.nn.relu),
              Conv2D(self.channel_n, 1, activation=None,
                  kernel_initializer=tf.zeros_initializer),
        ])

        self(tf.zeros([1, 3, 3, channel_n]))  # dummy call to build the model

    @tf.function
    def perceive(self, x, angle=0.0):
        identify = np.float32([0, 1, 0])
        identify = np.outer(identify, identify)
        dx = np.outer([1, 2, 1], [-1, 0, 1]) / 8.0  # Sobel filter
        dy = dx.T
        c, s = tf.cos(angle), tf.sin(angle)
        kernel = tf.stack([identify, c*dx-s*dy, s*dx+c*dy], -1)[:, :, None, :]
        kernel = tf.repeat(kernel, self.channel_n, 2)
        y = tf.nn.depthwise_conv2d(x, kernel, [1, 1, 1, 1], 'SAME')
        return y

    @tf.function
    def call(self, x, fire_rate=None, angle=0.0, step_size=1.0):
        pre_life_mask = get_living_mask(x)

        y = self.perceive(x, angle)
        dx = self.dmodel(y)*step_size
        if fire_rate is None:
            fire_rate = self.fire_rate
        update_mask = tf.random.uniform(tf.shape(x[:, :, :, :1])) <= fire_rate
        x += dx * tf.cast(update_mask, tf.float32)

        post_life_mask = get_living_mask(x)
        life_mask = pre_life_mask & post_life_mask
        return x * tf.cast(life_mask, tf.float32)


In [63]:
ca_model = CAModel(channel_n=16,fire_rate=0.5)

In [64]:
ca_model.dmodel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (1, 3, 3, 128)            6272      
_________________________________________________________________
conv2d_9 (Conv2D)            (1, 3, 3, 16)             2064      
Total params: 8,336
Trainable params: 8,336
Non-trainable params: 0
_________________________________________________________________


In [65]:
np.random.random

<function RandomState.random>

In [66]:
seed = np.random.random((72,72,16)).astype('f')

x0 = np.repeat(seed[None, ...], 8, 0)

In [67]:
x0.shape

(8, 72, 72, 16)

In [82]:
fire_rate=None
angle=0.0
step_size=1.0

pre_life_mask = get_living_mask(x0)
y = ca_model.perceive(x0, 0.0)
dx = ca_model.dmodel(y)*step_size
if fire_rate is None:
    fire_rate = ca_model.fire_rate
update_mask = tf.random.uniform(tf.shape(x0[:, :, :, :1])) <= fire_rate
x0 += dx * tf.cast(update_mask, tf.float32)

post_life_mask = get_living_mask(x0)
life_mask = pre_life_mask & post_life_mask
return_var = x0 * tf.cast(life_mask, tf.float32)


In [68]:
y = ca_model.perceive(x0)

In [69]:
dx = ca_model.dmodel(y) 

In [70]:
y.shape

TensorShape([8, 72, 72, 48])

In [71]:
dx.shape

TensorShape([8, 72, 72, 16])

In [72]:
angle = 0.0
identify = np.float32([0, 1, 0])
identify = np.outer(identify, identify)
dx = np.outer([1, 2, 1], [-1, 0, 1]) / 8.0  # Sobel filter
dy = dx.T
c, s = tf.cos(angle), tf.sin(angle)
kernel1 = tf.stack([identify, c*dx-s*dy, s*dx+c*dy], -1)[:, :, None, :]
kernel = tf.repeat(kernel1, 16, 2)

In [73]:
identify

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]], dtype=float32)

In [74]:
c*dx-s*dy

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.125,  0.   ,  0.125],
       [-0.25 ,  0.   ,  0.25 ],
       [-0.125,  0.   ,  0.125]], dtype=float32)>

In [75]:
s*dx+c*dy

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.125, -0.25 , -0.125],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.125,  0.25 ,  0.125]], dtype=float32)>

In [76]:
dx

array([[-0.125,  0.   ,  0.125],
       [-0.25 ,  0.   ,  0.25 ],
       [-0.125,  0.   ,  0.125]])

In [77]:
dy

array([[-0.125, -0.25 , -0.125],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.125,  0.25 ,  0.125]])

In [78]:
c

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [79]:
s

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [80]:
kernel1

<tf.Tensor: shape=(3, 3, 1, 3), dtype=float32, numpy=
array([[[[ 0.   , -0.125, -0.125]],

        [[ 0.   ,  0.   , -0.25 ]],

        [[ 0.   ,  0.125, -0.125]]],


       [[[ 0.   , -0.25 ,  0.   ]],

        [[ 1.   ,  0.   ,  0.   ]],

        [[ 0.   ,  0.25 ,  0.   ]]],


       [[[ 0.   , -0.125,  0.125]],

        [[ 0.   ,  0.   ,  0.25 ]],

        [[ 0.   ,  0.125,  0.125]]]], dtype=float32)>

In [45]:
kernel.shape

TensorShape([3, 3, 16, 3])